# This notebook processes pig recombination rate maps from papers for inclusion into `stdpopsim`

The target HapMap format used by `stdpopsim` is described in [msprime.RateMap.read_hapmap](https://tskit.dev/msprime/docs/stable/api.html#msprime.RateMap.read_hapmap)

## Johnsson_2021

Directory `Johnsson_2021` contains the data from the supplement of this [paper](https://doi.org/10.1186/s12711-021-00643-0). Specifically, the supplement has recombination rate maps in 1-Mbp windows for:
  * males (Additional file 2: Table S1),
  * females (Additional file 3: Table S2), and
  * sex-averaged (Additional file 4: Table S3).

Here, we use only sex-averaged map. We downloaded the data using the following bash shell commands:


In [1]:
%%bash
set -euo pipefail
mkdir -p Johnsson_2021/Johnsson_2021_sex-averaged_rec_map
cd Johnsson_2021
wget https://static-content.springer.com/esm/art%3A10.1186%2Fs12711-021-00643-0/MediaObjects/12711_2021_643_MOESM4_ESM.csv

--2026-02-19 08:06:11--  https://static-content.springer.com/esm/art%3A10.1186%2Fs12711-021-00643-0/MediaObjects/12711_2021_643_MOESM4_ESM.csv
Resolving static-content.springer.com (static-content.springer.com)... 151.101.236.95
Connecting to static-content.springer.com (static-content.springer.com)|151.101.236.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 91337 (89K) [application/octet-stream]
Saving to: ‘12711_2021_643_MOESM4_ESM.csv.1’

     0K .......... .......... .......... .......... .......... 56% 7.59M 0s
    50K .......... .......... .......... .........            100% 29.6M=0.008s

2026-02-19 08:06:11 (11.3 MB/s) - ‘12711_2021_643_MOESM4_ESM.csv.1’ saved [91337/91337]



We can now move to formatting!

In [2]:
# Load packages
import msprime
import pandas as pd
import numpy as np

In [3]:
# Import the data
df = pd.read_csv(
    "Johnsson_2021/12711_2021_643_MOESM4_ESM.csv",
    sep=",",
    header=0,
)
print(df)

      chr  window_start  window_end  recombination_rate
0       1             1     1000000            0.009154
1       1       1000001     2000000            0.009496
2       1       2000001     3000000            0.006806
3       1       3000001     4000000            0.011100
4       1       4000001     5000000            0.015028
...   ...           ...         ...                 ...
2267   18      51000001    52000000            0.018101
2268   18      52000001    53000000            0.010463
2269   18      53000001    54000000            0.008005
2270   18      54000001    55000000            0.010662
2271   18      55000001    55807287            0.000000

[2272 rows x 4 columns]


In [4]:
# Calculate average recombination rate for each chromosome
# This is used in stdpopsim for the pig species genome definitions
# (for the chromosome-specific recombination rate)
chr_length = []
chr_rec_rate_mean = []
chr_rec_rate_sum = []
for chr in range(1, 19):
    # chr = 1
    chr_data = df.groupby("chr").get_group(chr)
    rec_rate = (chr_data["recombination_rate"] * 1e-6).tolist()
    # * 1e-6 because rates are Morgans per 1 Mbp window
    pos = [0]
    pos.extend(chr_data["window_end"].tolist())
    rate_map = msprime.RateMap(position=pos, rate=rec_rate)
    this_chr_length = rate_map.position[-1]
    chr_length.append(this_chr_length)
    chr_rec_rate_mean.append(rate_map.mean_rate)
    chr_rec_rate_sum.append(sum(chr_data["recombination_rate"]))
    # rate_map.mean_rate gives equivalent result to sum(chr_data["recombination_rate"]) / this_chr_length)

print("Index Chromosome Length(bp) Length(M) Rate(M/bp)")
for i in range(0, 18):
    print(
        i,
        i + 1,
        chr_length[i],
        round(chr_rec_rate_sum[i], 2),
        round(chr_rec_rate_mean[i], 11),
    )

print("\nTotal length(bp):", sum(chr_length))
print("\nTotal length(M):", sum(chr_rec_rate_sum))
print("\nTotal rate(M/bp):", sum(chr_rec_rate_sum) / sum(chr_length))

Index Chromosome Length(bp) Length(M) Rate(M/bp)
0 1 274287862.0 1.46 5.34e-09
1 2 151917421.0 1.31 8.61e-09
2 3 132700427.0 1.29 9.75e-09
3 4 130900910.0 1.28 9.79e-09
4 5 104477606.0 1.25 1.199e-08
5 6 170825645.0 1.5 8.79e-09
6 7 121782508.0 1.33 1.089e-08
7 8 138930735.0 1.24 8.92e-09
8 9 139401727.0 1.3 9.34e-09
9 10 69319537.0 1.15 1.654e-08
10 11 79072521.0 0.96 1.215e-08
11 12 61598258.0 1.04 1.691e-08
12 13 208240759.0 1.28 6.16e-09
13 14 141719266.0 1.24 8.72e-09
14 15 140404164.0 1.14 8.11e-09
15 16 79449474.0 0.91 1.141e-08
16 17 63451800.0 0.87 1.378e-08
17 18 55807287.0 0.76 1.368e-08

Total length(bp): 2264287907.0

Total length(M): 21.31916806136578

Total rate(M/bp): 9.415396335182467e-09


FYI The chromosome and total lengths are not the same as in
https://popsim-consortium.github.io/stdpopsim-docs/stable/catalog.html#sec_catalog_SusScr_genome
which are from https://www.ensembl.org/Sus_scrofa 11.1 & https://www.ebi.ac.uk/ena/browser/view/GCA_000003025.6
This is OK, since we don't directly use the total chromosome lengths provided here
(we can have a map with no data at the start and end of chromosomes).
Looking at how human genome recombination rate maps were done,
they also have a missing end part for each (or at least many chromosomes);
I have seen this by comparing map ends in https://stdpopsim.s3-us-west-2.amazonaws.com/genetic_maps/HomSap/HapMapII_GRCh38.tar.gz
with chr ends at https://popsim-consortium.github.io/stdpopsim-docs/stable/catalog.html#sec_catalog_HomSap.

In [5]:
for chr in range(1, 19):
    # chr = 1
    chr_data = df.groupby("chr").get_group(chr)
    physical_position_previous = chr_data["window_start"].tolist()
    physical_position = chr_data["window_end"].tolist()
    n = len(physical_position)
    physical_position.extend([physical_position[-1]])
    rec_rate = (chr_data["recombination_rate"] * 1e-6).tolist()
    # * 1e-6 because original rate is per Mbp
    rec_rate.extend([0.0])
    cM_per_region = [0.0] * n
    # cM_per_region will hold cM/region (next interval) and not simply cM/Mbp;
    # though here they are the same since regions are 1Mbp long
    # (except the last one).
    genetic_position = [0.0] * n
    for region in range(n):
        if region == 0:
            tmp = 0.0
        else:
            tmp = genetic_position[region - 1]
        delta_p_previous = (
            physical_position[region] - physical_position_previous[region] + 1
        )
        cM_per_region_previous = rec_rate[region] * delta_p_previous * 100
        genetic_position[region] = tmp + cM_per_region_previous
        delta_p_current = (
            physical_position[region + 1] - physical_position[region]
        )  # no need for +1 here
        cM_per_region_current = rec_rate[region + 1] * delta_p_current * 100
        cM_per_region[region] = cM_per_region_current
        # print(
        #    region,
        #    physical_position_previous[region],
        #    physical_position[region],
        #    delta_p_current,
        #    rec_rate[region],
        #    cM_per_region[region],
        #    genetic_position[region],
        # )

    # Create DataFrame,
    genetic_map_df = pd.DataFrame(
        {
            "Chromosome": f"chr{chr}",
            "Position(bp)": [0] + physical_position[:n],
            "Rate(cM/Mb)": [genetic_position[0]] + cM_per_region,
            "Map(cM)": [0] + genetic_position,
        }
    )
    genetic_map_df["Position(bp)"] = genetic_map_df["Position(bp)"].astype(int)
    genetic_map_df.to_csv(
        f"Johnsson_2021/Johnsson_2021_sex-averaged_rec_map/Johnsson_2021_sex-averaged_rec_map_chr{chr}.txt",
        sep="\t",
        index=False,
    )

In [6]:
chr1 = msprime.RateMap.read_hapmap(
    fileobj="Johnsson_2021/Johnsson_2021_sex-averaged_rec_map/Johnsson_2021_sex-averaged_rec_map_chr1.txt",
    rate_col=None,
    map_col=3,
)
print(chr1)


┌───────────────────────────────────────────────────────┐
│left       │right      │        mid│     span│     rate│
├───────────────────────────────────────────────────────┤
│0          │1000000    │     500000│  1000000│  9.2e-09│
│1000000    │2000000    │    1500000│  1000000│  9.5e-09│
│2000000    │3000000    │    2500000│  1000000│  6.8e-09│
│3000000    │4000000    │    3500000│  1000000│  1.1e-08│
│4000000    │5000000    │    4500000│  1000000│  1.5e-08│
│5000000    │6000000    │    5500000│  1000000│    3e-08│
│6000000    │7000000    │    6500000│  1000000│  2.2e-08│
│7000000    │8000000    │    7500000│  1000000│  1.5e-08│
│8000000    │9000000    │    8500000│  1000000│    2e-08│
│9000000    │10000000   │    9500000│  1000000│  1.4e-08│
│⋯          │⋯          │          ⋯│        ⋯│        ⋯│
│265000000  │266000000  │  265500000│  1000000│  1.4e-08│
│266000000  │267000000  │  266500000│  1000000│  2.1e-08│
│267000000  │268000000  │  267500000│  1000000│  2.7e-08│
│268000000  │

In [7]:
chr1 = msprime.RateMap.read_hapmap(
    fileobj="./Johnsson_2021/Johnsson_2021_sex-averaged_rec_map/Johnsson_2021_sex-averaged_rec_map_chr1.txt",
    rate_col=2,
    map_col=None,
)
print(chr1)


┌───────────────────────────────────────────────────────┐
│left       │right      │        mid│     span│     rate│
├───────────────────────────────────────────────────────┤
│0          │1000000    │     500000│  1000000│  9.2e-09│
│1000000    │2000000    │    1500000│  1000000│  9.5e-09│
│2000000    │3000000    │    2500000│  1000000│  6.8e-09│
│3000000    │4000000    │    3500000│  1000000│  1.1e-08│
│4000000    │5000000    │    4500000│  1000000│  1.5e-08│
│5000000    │6000000    │    5500000│  1000000│    3e-08│
│6000000    │7000000    │    6500000│  1000000│  2.2e-08│
│7000000    │8000000    │    7500000│  1000000│  1.5e-08│
│8000000    │9000000    │    8500000│  1000000│    2e-08│
│9000000    │10000000   │    9500000│  1000000│  1.4e-08│
│⋯          │⋯          │          ⋯│        ⋯│        ⋯│
│265000000  │266000000  │  265500000│  1000000│  1.4e-08│
│266000000  │267000000  │  266500000│  1000000│  2.1e-08│
│267000000  │268000000  │  267500000│  1000000│  2.7e-08│
│268000000  │

In [8]:
!tar -czvf Johnsson_2021/Johnsson_2021_sex-averaged_rec_map.tar.gz Johnsson_2021/Johnsson_2021_sex-averaged_rec_map

a Johnsson_2021/Johnsson_2021_sex-averaged_rec_map
a Johnsson_2021/Johnsson_2021_sex-averaged_rec_map/Johnsson_2021_sex-averaged_rec_map_chr9.txt
a Johnsson_2021/Johnsson_2021_sex-averaged_rec_map/Johnsson_2021_sex-averaged_rec_map_chr8.txt
a Johnsson_2021/Johnsson_2021_sex-averaged_rec_map/Johnsson_2021_sex-averaged_rec_map_chr18.txt
a Johnsson_2021/Johnsson_2021_sex-averaged_rec_map/Johnsson_2021_sex-averaged_rec_map_chr12.txt
a Johnsson_2021/Johnsson_2021_sex-averaged_rec_map/Johnsson_2021_sex-averaged_rec_map_chr13.txt
a Johnsson_2021/Johnsson_2021_sex-averaged_rec_map/Johnsson_2021_sex-averaged_rec_map_chr11.txt
a Johnsson_2021/Johnsson_2021_sex-averaged_rec_map/Johnsson_2021_sex-averaged_rec_map_chr10.txt
a Johnsson_2021/Johnsson_2021_sex-averaged_rec_map/Johnsson_2021_sex-averaged_rec_map_chr14.txt
a Johnsson_2021/Johnsson_2021_sex-averaged_rec_map/Johnsson_2021_sex-averaged_rec_map_chr15.txt
a Johnsson_2021/Johnsson_2021_sex-averaged_rec_map/Johnsson_2021_sex-averaged_rec_map_c

## Brekke_2022

Directory `Brekke_2022` contains the data from this [paper](https://doi.org/10.1186/s12711-022-00723-9). Specifically, we contacted the lead author who shared the `Linkagemap_SusScrofa_*.txt` files, where `*` is `Duroc`, `Landrace`, `LargeWhite`, `Pietrain`, and `Synthetic`. The files contain sex-averaged, male, and female map. Here we only process sex-averaged map, but could late process also sex-specific maps.

In [9]:
%%bash
set -euo pipefail
mkdir -p Brekke_2022/Brekke_2022_Duroc_sex-averaged_rec_map
mkdir -p Brekke_2022/Brekke_2022_Landrace_sex-averaged_rec_map
mkdir -p Brekke_2022/Brekke_2022_LargeWhite_sex-averaged_rec_map
mkdir -p Brekke_2022/Brekke_2022_Pietrain_sex-averaged_rec_map
mkdir -p Brekke_2022/Brekke_2022_Synthetic_sex-averaged_rec_map


In [10]:
def fun_rec_brekke(file, output):
    df = pd.read_csv(file, sep=" ", header=0)
    for chr in range(1, 19):
        chr_data = df.groupby("chr").get_group(chr)
        position = chr_data["bp"].tolist()
        Map = chr_data["cM"].tolist()
        cM_per_region = [Map[0] / (position[0] - 1) * 1e6]
        # Adjust for 0-based index
        # *1e6 to convert from cM/bp to cM/Mb
        physical_position = [0]  # Start with position 0
        genetic_position = [0.0]  # Start with genetic position 0
        for region in range(len(Map)):
            if region < len(Map) - 1:
                physical_position.append(
                    position[region] - 1
                )  # Adjust for 0-based index
                genetic_position.append(Map[region])
                delta_p = position[region + 1] - position[region]
                delta_g = Map[region + 1] - Map[region]
                cM_per_region.append(delta_g / delta_p * 1e6)
            else:
                # last rate could be missing;
                # to avoid NaN, we use rate=0 for the last region
                physical_position.append(
                    position[region] - 1
                )  # Adjust for 0-based index
                genetic_position.append(Map[region])
                cM_per_region.append(0.0)

        # Create DataFrame for genetic map
        genetic_map_df = pd.DataFrame(
            {
                "Chromosome": f"chr{chr}",
                "Position(bp)": physical_position,
                "Rate(cM/Mb)": cM_per_region,
                "Map(cM)": genetic_position,
            }
        )
        genetic_map_df["Position(bp)"] = genetic_map_df["Position(bp)"].astype(int)
        genetic_map_df.to_csv(f"{output}_chr{chr}.txt", sep="\t", index=False)

In [11]:
fun_rec_brekke(
    "Brekke_2022/Linkagemap_SusScrofa_Duroc.txt",
    "Brekke_2022/Brekke_2022_Duroc_sex-averaged_rec_map/Brekke_2022_Duroc_sex-averaged_rec_map",
)
fun_rec_brekke(
    "Brekke_2022/Linkagemap_SusScrofa_Landrace.txt",
    "Brekke_2022/Brekke_2022_Landrace_sex-averaged_rec_map/Brekke_2022_Landrace_rec_map",
)
fun_rec_brekke(
    "Brekke_2022/Linkagemap_SusScrofa_LargeWhite.txt",
    "Brekke_2022/Brekke_2022_LargeWhite_sex-averaged_rec_map/Brekke_2022_LargeWhite_rec_map",
)
fun_rec_brekke(
    "Brekke_2022/Linkagemap_SusScrofa_Pietrain.txt",
    "Brekke_2022/Brekke_2022_Pietrain_sex-averaged_rec_map/Brekke_2022_Pietrain_rec_map",
)
fun_rec_brekke(
    "Brekke_2022/Linkagemap_SusScrofa_Synthetic.txt",
    "Brekke_2022/Brekke_2022_Synthetic_sex-averaged_rec_map/Brekke_2022_Synthetic_rec_map",
)

In [12]:
chr1 = msprime.RateMap.read_hapmap(
    fileobj="Brekke_2022/Brekke_2022_Duroc_sex-averaged_rec_map/Brekke_2022_Duroc_sex-averaged_rec_map_chr1.txt",
    rate_col=None,
    map_col=3,
)
print(chr1)
print(chr1.mean_rate)


┌────────────────────────────────────────────────────────┐
│left       │right      │          mid│    span│     rate│
├────────────────────────────────────────────────────────┤
│0          │161012     │        80506│  161012│        0│
│161012     │184557     │     172784.5│   23545│        0│
│184557     │239522     │     212039.5│   54965│  1.8e-08│
│239522     │261793     │     250657.5│   22271│        0│
│261793     │289362     │     275577.5│   27569│        0│
│289362     │335212     │       312287│   45850│        0│
│335212     │381074     │       358143│   45862│        0│
│381074     │408639     │     394856.5│   27565│        0│
│408639     │422681     │       415660│   14042│  3.3e-08│
│422681     │437278     │     429979.5│   14597│        0│
│⋯          │⋯          │            ⋯│       ⋯│        ⋯│
│273956006  │274049404  │    274002705│   93398│  1.4e-08│
│274049404  │274055414  │    274052409│    6010│  5.6e-08│
│274055414  │274093225  │  274074319.5│   37811│  1.4e-

In [13]:
chr1 = msprime.RateMap.read_hapmap(
    fileobj="Brekke_2022/Brekke_2022_Duroc_sex-averaged_rec_map/Brekke_2022_Duroc_sex-averaged_rec_map_chr1.txt",
    rate_col=2,
    map_col=None,
)
print(chr1)
print(chr1.mean_rate)


┌────────────────────────────────────────────────────────┐
│left       │right      │          mid│    span│     rate│
├────────────────────────────────────────────────────────┤
│0          │161012     │        80506│  161012│        0│
│161012     │184557     │     172784.5│   23545│        0│
│184557     │239522     │     212039.5│   54965│  1.8e-08│
│239522     │261793     │     250657.5│   22271│        0│
│261793     │289362     │     275577.5│   27569│        0│
│289362     │335212     │       312287│   45850│        0│
│335212     │381074     │       358143│   45862│        0│
│381074     │408639     │     394856.5│   27565│        0│
│408639     │422681     │       415660│   14042│  3.3e-08│
│422681     │437278     │     429979.5│   14597│        0│
│⋯          │⋯          │            ⋯│       ⋯│        ⋯│
│273956006  │274049404  │    274002705│   93398│  1.4e-08│
│274049404  │274055414  │    274052409│    6010│  5.6e-08│
│274055414  │274093225  │  274074319.5│   37811│  1.4e-

In [15]:
!tar -czvf Brekke_2022/Brekke_2022_Duroc_sex-averaged_rec_map.tar.gz      Brekke_2022/Brekke_2022_Duroc_sex-averaged_rec_map
!tar -czvf Brekke_2022/Brekke_2022_Landrace_sex-averaged_rec_map.tar.gz   Brekke_2022/Brekke_2022_Landrace_sex-averaged_rec_map
!tar -czvf Brekke_2022/Brekke_2022_LargeWhite_sex-averaged_rec_map.tar.gz Brekke_2022/Brekke_2022_LargeWhite_sex-averaged_rec_map
!tar -czvf Brekke_2022/Brekke_2022_Pietrain_sex-averaged_rec_map.tar.gz   Brekke_2022/Brekke_2022_Pietrain_sex-averaged_rec_map
!tar -czvf Brekke_2022/Brekke_2022_synthetic_sex-averaged_rec_map.tar.gz  Brekke_2022/Brekke_2022_synthetic_sex-averaged_rec_map

a Brekke_2022/Brekke_2022_Duroc_sex-averaged_rec_map
a Brekke_2022/Brekke_2022_Duroc_sex-averaged_rec_map/Brekke_2022_Duroc_sex-averaged_rec_map_chr5.txt
a Brekke_2022/Brekke_2022_Duroc_sex-averaged_rec_map/Brekke_2022_Duroc_sex-averaged_rec_map_chr4.txt
a Brekke_2022/Brekke_2022_Duroc_sex-averaged_rec_map/Brekke_2022_Duroc_sex-averaged_rec_map_chr6.txt
a Brekke_2022/Brekke_2022_Duroc_sex-averaged_rec_map/Brekke_2022_Duroc_sex-averaged_rec_map_chr7.txt
a Brekke_2022/Brekke_2022_Duroc_sex-averaged_rec_map/Brekke_2022_Duroc_sex-averaged_rec_map_chr18.txt
a Brekke_2022/Brekke_2022_Duroc_sex-averaged_rec_map/Brekke_2022_Duroc_sex-averaged_rec_map_chr3.txt
a Brekke_2022/Brekke_2022_Duroc_sex-averaged_rec_map/Brekke_2022_Duroc_sex-averaged_rec_map_chr2.txt
a Brekke_2022/Brekke_2022_Duroc_sex-averaged_rec_map/Brekke_2022_Duroc_sex-averaged_rec_map_chr1.txt
a Brekke_2022/Brekke_2022_Duroc_sex-averaged_rec_map/Brekke_2022_Duroc_sex-averaged_rec_map_chr17.txt
a Brekke_2022/Brekke_2022_Duroc_sex-